In [ ]:
import pandas as pd
df = pd.read_csv("/content/sentiment_analysis.csv",encoding="latin1")

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

Ah, yep! That error means the GloVe file isn't in your working directory yet — specifically glove.6B.100d.txt is missing. No worries, just run this quick setup and you'll be good to go 👇

✅ One-Time Setup: Download and Extract GloVe
Paste this into a new code cell before loading the GloVe file:

python
Copy code


In [ ]:
# One-time download and unzip
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2025-04-14 18:28:22--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-14 18:28:22--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-14 18:28:23--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [ ]:
!ls

glove.6B.100d.txt  glove.6B.50d.txt  label_encoder.pkl	     sentiment_model.h5
glove.6B.200d.txt  glove.6B.zip      sample_data	     tokenizer.pkl
glove.6B.300d.txt  glove.6B.zip.1    sentiment_analysis.csv


In [ ]:
print(df.columns)

Index(['neutral', 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .'], dtype='object')


In [ ]:
df.shape

(4845, 2)

In [ ]:
#start here

practice


In [ ]:
# STEP 0: One-time download and unzip GloVe (300d)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

# STEP 1: Imports
import pandas as pd
import numpy as np
import re
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# STEP 2: Load Data
df = pd.read_csv("/content/sentiment_analysis.csv", encoding="latin1")
df.columns = ["labels", "text"]
df.dropna(subset=["labels", "text"], inplace=True)
df = df[df["labels"].astype(str).str.strip() != ""]

# STEP 3: Clean Text
def clean_text(text):
    text = re.sub(r'<.*?>', '', str(text))
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

df["clean_text"] = df["text"].apply(clean_text)

# STEP 4: Encode Labels
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["labels"])
y = to_categorical(df["label_encoded"])

# STEP 5: Tokenize and Pad
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])
sequences = tokenizer.texts_to_sequences(df["clean_text"])
max_len = 100
X = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")

# STEP 6: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# STEP 7: Load GloVe 300D Embeddings
glove_path = "/content/glove.6B.300d.txt"
embedding_index = {}
with open(glove_path, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
print(f"✅ Loaded {len(embedding_index)} word vectors from GloVe.")

# STEP 8: Create Embedding Matrix
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# STEP 9: Build Model
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_len,
              trainable=True),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

# STEP 10: Train Model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(df["label_encoded"]),
                                     y=df["label_encoded"])
class_weights_dict = dict(enumerate(class_weights))

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=2
)

# STEP 11: Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n✅ Final Test Accuracy: {accuracy:.4f}")

# STEP 12: Classification Report
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)
print("\n🧾 Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# STEP 13: Save Model
model.save("/content/sentiment_model.h5")
with open("/content/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("/content/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)
print("✅ Model, tokenizer, and label encoder saved successfully!")

# STEP 14: Predict Function
def predict_user_input():
    while True:
        user_text = input("\n💬 Enter a sentence (or type 'exit' to stop): ")
        if user_text.lower() == 'exit':
            break
        cleaned = clean_text(user_text)
        sequence = tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
        prediction = model.predict(padded)
        label_index = np.argmax(prediction, axis=1)[0]
        sentiment = label_encoder.inverse_transform([label_index])[0]
        print(f"🔮 Predicted Sentiment: {sentiment}")

# 🔥 Call this function to test:
# predict_user_input()


--2025-04-14 18:31:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-04-14 18:31:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-04-14 18:31:49--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

gl

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │       3,374,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,374,100 (12.87 MB)

 Trainable params: 3,374,100 (12.87 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
109/109 - 8s - 73ms/step - accuracy: 0.5390 - loss: 0.9448 - val_accuracy: 0.5335 - val_loss: 0.9960
Epoch 2/20
109/109 - 2s - 19ms/step - accuracy: 0.7342 - loss: 0.6262 - val_accuracy: 0.7113 - val_loss: 0.7419
Epoch 3/20
109/109 - 3s - 24ms/step - accuracy: 0.8581 - loss: 0.3624 - val_accuracy: 0.7448 - val_loss: 0.6752
Epoch 4/20
109/109 - 3s - 30ms/step - accuracy: 0.9309 - loss: 0.1909 - val_accuracy: 0.7629 - val_loss: 0.7727
Epoch 5/20
109/109 - 2s - 22ms/step - accuracy: 0.9581 - loss: 0.1274 - val_accuracy: 0.7423 - val_loss: 0.9378
Epoch 6/20
109/109 - 2s - 20ms/step - accuracy: 0.9756 - loss: 0.0695 - val_accuracy: 0.7758 - val_loss: 0.8871
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7358 - loss: 0.6828

✅ Final Test Accuracy: 0.7523
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step



🧾 Classification Report:

              precision    recall  f1-score   support

    negative       0.68      0.71      0.69       115
     neutral       0.82      0.79      0.81       567
    positive       0.66      0.69      0.68       287

    accuracy                           0.75       969
   macro avg       0.72      0.73      0.73       969
weighted avg       0.76      0.75      0.75       969

✅ Model, tokenizer, and label encoder saved successfully!


Add this to save the model and tokenizer

In [ ]:
import pickle

# STEP 14: Save the model
model.save("/content/sentiment_model.h5")

# Save the tokenizer
with open("/content/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save the label encoder
with open("/content/label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ Model, tokenizer, and label encoder saved successfully!")


✅ Model, tokenizer, and label encoder saved successfully!


 Later, to load and reuse them

In [ ]:
from tensorflow.keras.models import load_model
import pickle

# Load the model
model = load_model("/content/sentiment_model.h5")

# Load the tokenizer
with open("/content/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# Load the label encoder
with open("/content/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

print("✅ Model, tokenizer, and label encoder loaded!")


✅ Model, tokenizer, and label encoder loaded!


In [ ]:
def predict_user_input():
    while True:
        user_text = input("\n💬 Enter a sentence (or type 'exit' to stop): ")
        if user_text.lower() == 'exit':
            break

        # Clean the input
        cleaned = clean_text(user_text)

        # Tokenize and pad
        sequence = tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

        # Predict
        prediction = model.predict(padded)
        label_index = np.argmax(prediction, axis=1)[0]
        sentiment = label_encoder.inverse_transform([label_index])[0]

        print(f"🔮 Predicted Sentiment: {sentiment}")


In [ ]:
predict_user_input()


💬 Enter a sentence (or type 'exit' to stop): "The company reported a significant increase in quarterly profits."
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🔮 Predicted Sentiment: positive

💬 Enter a sentence (or type 'exit' to stop):  "Shares surged after the announcement of a strategic partnership."
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🔮 Predicted Sentiment: positive

💬 Enter a sentence (or type 'exit' to stop): "Investors are optimistic about the upcoming product launch."
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
🔮 Predicted Sentiment: neutral

💬 Enter a sentence (or type 'exit' to stop):  "Revenue growth exceeded analysts' expectations."
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🔮 Predicted Sentiment: positive

💬 Enter a sentence (or type 'exit' to stop): "The CEO's vision has been praised by market analysts.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
🔮 Predicted Sentiment: positive

💬 Enter a sentence (or type 'exit' to stop): "The firm released its third-quarter earnings report today."
1/1 ━━━━━━